# Data Cleaning for A1 - Data Visualisation

## 1. Setting up

In [1]:
##### 2. ATP Player Information #####
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
base_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_players.csv"
player_info = pd.read_csv(base_url)

/var/folders/ph/fn7b7pn94t937623__53rx0c0000gn/T/ipykernel_38117/318463326.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  player_info = pd.read_csv(base_url)


# A. Prepare Data for Gantt Chart

In [2]:
##########################################
'''
The data ranking player files includes the rankings of all players regardless of when they retired. 
This leads to issues on the Gantt Chart - namely that retired players misleadingly appear active on tour.

To address this issue, select players were chosen to be displayed on the Gantt Chart,
and their retirement years were manually researched and added in.
'''
##########################################

past_gen_au = ["Rod Laver", "Ken Rosewall", "Roy Emerson", "Pat Rafter", "Lleyton Hewitt"]

current_gen_au = ["Adam Walton", "Aleksandar Vukic", "Alex De Minaur", "Alexei Popyrin", 
              "Christopher Oconnell", "James Duckworth", "Jordan Thompson", "Lleyton Hewitt", 
              "Nick Kyrgios", "Rinky Hijikata", "Thanasi Kokkinakis", "Tristan Schoolkate"]

big_four = ["Novak Djokovic", "Roger Federer", "Rafael Nadal", "Andy Murray"]

top_current_players = ["Carlos Alcaraz", "Jannik Sinner", "Alexander Zverev", "Daniil Medvedev"]

# End of career info
end_year_data = {
    "Player": [
        "Adam Walton", "Aleksandar Vukic", "Alex De Minaur", "Alexei Popyrin",
        "Christopher Oconnell", "James Duckworth", "Jordan Thompson", 
        "Lleyton Hewitt", "Nick Kyrgios", "Rinky Hijikata", 
        "Thanasi Kokkinakis", "Tristan Schoolkate", "Rod Laver",
        "Ken Rosewall", "Roy Emerson", "Pat Rafter",
        "Novak Djokovic", "Roger Federer", "Rafael Nadal", "Andy Murray",
        "Carlos Alcaraz", "Jannik Sinner", "Alexander Zverev", "Daniil Medvedev"
    ],
    "End of Career Year": [
        None, None, None, None, 
        None, None, None, 
        2016, None, None, 
        None, None, 1979,
        1977, 1980, 2003,
        None, 2022, 2024, 2024,
        None, None, None, None
    ],
    "Country Code": [
        "Australia", "Australia", "Australia", "Australia", 
        "Australia", "Australia", "Australia",
        "Australia", "Australia", "Australia",
        "Australia", "Australia", "Australia",
        "Australia", "Australia", "Australia",
        "Serbia", "Switzerland", "Spain", "Britain",
        "Spain", "Italy", "Germany", "Russia"
    ],
    "Filter Category": [
        "Top Active AU player", "Top Active AU player", "Top Active AU player", "Top Active AU player",
        "Top Active AU player", "Top Active AU player", "Top Active AU player",
        "Top Past AU player", "Top Active AU player", "Top Active AU player",
         "Top Active AU player", "Top Active AU player", "Top Past AU player",
         "Top Past AU player", "Top Past AU player", "Top Past AU player",
         "Big Four", "Big Four", "Big Four", "Big Four",
         "Top Active player", "Top Active player", "Top Active player", "Top Active player"
    ]

}
end_career_table = pd.DataFrame(end_year_data)


In [3]:
# 1. Get Rankings Data
ranks = pd.read_csv("data/imputed_ranking_data.csv")
ranks["ranking_date"] = pd.to_datetime(ranks["ranking_date"])

# 2. Get Player Names and Rankings
player_ranks = ranks.merge(player_info, left_on="player", right_on="player_id", how="left")
player_ranks["player_name"] = player_ranks["name_first"] + " " + player_ranks["name_last"]

# 3. Filter to only AU players
filtered_players = player_ranks[player_ranks["player_name"].isin(past_gen_au + current_gen_au + big_four + top_current_players)]

# 4. Merge End of Career info into filtered_df
filtered_players = filtered_players.merge(end_career_table, how="left", left_on="player_name", right_on="Player")

# If End of Career Year is None, keep all entries (still active).
# Otherwise, filter out rows where ranking_date is after the End of Career Year
filtered_players = filtered_players[
    (filtered_players["End of Career Year"].isna()) | 
    (filtered_players["ranking_date"].dt.year <= filtered_players["End of Career Year"])
]

# Drop duplicate Player column if you don’t need it
filtered_players = filtered_players.drop(columns=["Player"])
# Save result
filtered_players.to_csv("final_tables_for_tableau/gantt_player_and_rankings.csv", index=False)

# B. Prepare Data for Radar Chart

In [2]:

##########################################
'''
M W%: % Matches won
TB W%: %Tie Breaks Won

Hld%: % Service Games Won       (Strength of Service Game)
1st%: % First Serve Points Won

Brk%: % Return Games Won        (Strength of Return Game)
v1st%: 1st Serve Return Points won      

                M W%
    Brk%                    Hld%
    v1st%                   1st%
                TB W%

'''
##########################################

# Load dataset
player_metrics = pd.read_csv("data/atp_100_advanced_player_info.csv")

# Select only relevant columns
relevant_player_metrics = ["Player", "M W%", "Hld%", "1st%", "TB W%", "v1st%", "Brk%"]
player_metrics = player_metrics[relevant_player_metrics]

# Convert percentage strings -> numeric values
for col in player_metrics.columns:
    if col != "Player":
        player_metrics[col] = player_metrics[col].str.rstrip('%').astype(float)

# ---- Compute averages across all metrics ----
averages = player_metrics.drop(columns=["Player"]).mean()

# Add as new row
average_row = pd.DataFrame([{"Player": "A - [AVERAGE]", **averages.to_dict()}])
player_metrics_with_avg = pd.concat([player_metrics, average_row], ignore_index=True)

# Convert back to % strings (2 decimals)
for col in player_metrics_with_avg.columns:
    if col != "Player":
        player_metrics_with_avg[col] = player_metrics_with_avg[col].round(2).astype(str) + "%"

# Pivot longer
player_metrics_long = pd.melt(
    player_metrics_with_avg,
    id_vars=["Player"],
    var_name="Metric",
    value_name="Value"
)

# Save
player_metrics_long.to_csv("final_tables_for_tableau/radar_player_metrics.csv", index=False)


## C. Multiple Line Chart (# Australian Players in the Top 100 of the ATP)

In [13]:
# 1. Prep Data
ranks = pd.read_csv("data/imputed_ranking_data.csv")
ranks["ranking_date"] = pd.to_datetime(ranks["ranking_date"])

merged = ranks.merge(player_info, left_on="player", right_on="player_id")         # Add Player Names to Rankings csv
merged["year"] = pd.to_datetime(merged["ranking_date"]).dt.year                   # Extract the year

# 2. Keep only Top 100 players on LAST ranking date of the year
top100 = merged[merged["rank"] <= 100]
last_day_per_year = top100.groupby("year")["ranking_date"].max().reset_index()
last_day_top100 = top100.merge(last_day_per_year, on=["year", "ranking_date"])    # Merge to keep only players in Top 100 on the last day of the year

# 3. After and including 1990, filter out all players where "points" column is Null
last_day_top100 = last_day_top100[(last_day_top100["points"].notna())]

#    For each ranking_date and rank, keep the player with the max points
idx = last_day_top100.groupby(["ranking_date", "rank"])["points"].idxmax()
last_day_top100 = last_day_top100.loc[idx].reset_index(drop=True)

# 4. Save 
last_day_top100.to_csv("final_tables_for_tableau/LineChart_top_players_byCountry.csv", index=False)

# 5. Check
yearly_sum = last_day_top100.groupby("year")["player_id"].nunique().reset_index(name="total_players")
yearly_sum.tail(3)

,year,total_players
32,2022,100
33,2023,100
34,2024,100


## D. Bar Chart on Tennis Adoption in Australia

In [21]:
# 1. Prep Data
playtennis_raw = pd.read_csv("data/au_tennis_participation.csv")

# Pivot longer (melt)
playtennis = playtennis_raw.melt(id_vars="Sport", var_name="Year", value_name="Percentage")

# Convert percentage strings to numeric floats
playtennis["Percentage"] = (
    playtennis["Percentage"]
    .str.replace("%", "", regex=False)  # remove %
    .astype(float)                      # convert to float
)

playtennis.to_csv("final_tables_for_tableau/barChart_au_tennis_participation.csv", index=False)


## E. Shape Scatter Plot on Earnings

In [30]:
import pandas as pd

# 1. Prep Data
playEarnings_raw = pd.read_csv("data/earnings.csv")                 # Earnings Data
ranks = pd.read_csv("data/today_atp100_ranks.csv")                  # Rankings Data

# Remove country codes in brackets from Player column
ranks["Player"] = (
    ranks["Player"]
    .str.replace(r"\(.*\)", "", regex=True)  # remove (XXX)
    .str.strip()                             # clean spaces
)

# 2. Preprocessing
playEarnings_raw = playEarnings_raw.drop_duplicates()   # Remove duplicates

# Convert money strings to numeric
playEarnings_raw["Total"] = (
    playEarnings_raw["Total"]
    .replace("[\$,]", "", regex=True)                   # remove $ and commas
    .astype(int)                                        # convert to int
)

playEarnings_raw["Year to Date"] = (
    playEarnings_raw["Year to Date"]
    .replace("[\$,]", "", regex=True)
    .astype(int)
)

# 3. Merge earnings with rank info
#    Standardize to lowercase for joining
playEarnings_raw["Player_clean"] = playEarnings_raw["Player"].str.lower().str.strip()
ranks["Player_clean"] = ranks["Player"].str.lower().str.strip()

#    Merge on cleaned names
earnings_with_ranks = playEarnings_raw.merge(
    ranks[["Player_clean", "Rank"]],
    on="Player_clean",
    how="left"
)

#    Restore proper capitalization (title case, e.g. "Jannik Sinner")
earnings_with_ranks["Player"] = earnings_with_ranks["Player_clean"].str.title()

#    Drop helper column
earnings_with_ranks = earnings_with_ranks.drop(columns=["Player_clean"])

#    Rename Year to Date → Earnings
earnings_with_ranks = earnings_with_ranks.rename(columns={"Year to Date": "Earnings"})

# 4. Feature Engineering
# 4a. Grand Slam Winner flag
grandslam_winners = [
    'Jannik Sinner', 'Carlos Alcaraz', 'Novak Djokovic',
    'Daniil Medvedev', 'Rafael Nadal', 'Roger Federer',
    'Andy Murray', 'Stan Wawrinka', 'Marin Cilic'
]
earnings_with_ranks["GrandSlamWinner"] = earnings_with_ranks["Player"].isin(grandslam_winners).astype(int)

# 4b. Add IOC (country) column
player_info["full_name"] = (player_info["name_first"] + " " + player_info["name_last"]).str.strip()

# Standardize for merging
earnings_with_ranks["Player_clean"] = earnings_with_ranks["Player"].str.lower().str.strip()
player_info["full_name_clean"] = player_info["full_name"].str.lower().str.strip()

# Merge
earnings_with_ranks = earnings_with_ranks.merge(
    player_info[["full_name_clean", "ioc"]],
    left_on="Player_clean",
    right_on="full_name_clean",
    how="left"
)

# Drop helper columns
earnings_with_ranks = earnings_with_ranks.drop(columns=["Player_clean", "full_name_clean"])

# 5. Save
earnings_with_ranks.to_csv("final_tables_for_tableau/shapeScatter_earnings.csv", index=False)


## G. Tree Diagram on Match Wins

In [50]:
# Load dataset
player_metrics = pd.read_csv("data/atp_100_advanced_player_info.csv")

# Select only relevant columns
relevant_player_metrics = ["Player", "M W%"]
player_metrics = player_metrics[relevant_player_metrics]

# Convert percentage strings -> numeric values
for col in player_metrics.columns:
    if col != "Player":
        player_metrics[col] = player_metrics[col].str.rstrip('%').astype(float)

# ---- Compute averages across all metrics ----
averages = player_metrics.drop(columns=["Player"]).mean()

# Add as new row
average_row = pd.DataFrame([{"Player": "A - [AVERAGE]", **averages.to_dict()}])
player_metrics_with_avg = pd.concat([player_metrics, average_row], ignore_index=True)

# Convert back to % strings (2 decimals)
for col in player_metrics_with_avg.columns:
    if col != "Player":
        player_metrics_with_avg[col] = player_metrics_with_avg[col].round(2).astype(str) + "%"

# Extract IOC country code from Player
player_metrics_with_avg["ioc"] = player_metrics_with_avg["Player"].str.extract(r"\[(.*?)\]")

player_metrics_with_avg["Player"] = (
    player_metrics_with_avg["Player"]
    .str.replace(r"\(.*\)", "", regex=True)   # e.g. (ESP)
    .str.replace(r"\[.*\]", "", regex=True)   # e.g. [ESP]
    .str.strip()
)

# Pivot longer
player_metrics_with_avg

#########################################################################################################

# Add Player Rank to each player
# Load rankings
ranks = pd.read_csv("data/today_atp100_ranks.csv")

# Clean Player names in ranks (remove country codes in brackets)
ranks["Player"] = (
    ranks["Player"]
    .str.replace(r"\(.*\)", "", regex=True)   # e.g. (ESP)
    .str.replace(r"\[.*\]", "", regex=True)   # e.g. [ESP]
    .str.strip()
)

# Standardize casing for join
player_metrics_with_avg["Player_clean"] = player_metrics_with_avg["Player"].str.lower().str.strip()
ranks["Player_clean"] = ranks["Player"].str.lower().str.strip()

# Merge
player_metrics_with_avg = player_metrics_with_avg.merge(
    ranks[["Player_clean", "Rank"]],
    on="Player_clean",
    how="left"
)

# Drop helper column
player_metrics_with_avg = player_metrics_with_avg.drop(columns=["Player_clean"])
player_metrics_with_avg
# Save
player_metrics_with_avg.to_csv("final_tables_for_tableau/treeDiagram_playerWinRates.csv", index=False)

## F. Scatterplot and Trend Line on Tennis Metrics

In [41]:
# Load dataset
player_metrics = pd.read_csv("data/atp_100_advanced_player_info.csv")

# Select only relevant columns
relevant_player_metrics = ["Player", "M W%", "Hld%", "1st%", "TB W%", "v1st%", "Brk%"]
player_metrics = player_metrics[relevant_player_metrics]

# Convert percentage strings -> numeric values
for col in player_metrics.columns:
    if col != "Player":
        player_metrics[col] = player_metrics[col].str.rstrip('%').astype(float)

# ---- Compute averages across all metrics ----
averages = player_metrics.drop(columns=["Player"]).mean()

# Add as new row
average_row = pd.DataFrame([{"Player": "A - [AVERAGE]", **averages.to_dict()}])
player_metrics_with_avg = pd.concat([player_metrics, average_row], ignore_index=True)

# Convert back to % strings (2 decimals)
for col in player_metrics_with_avg.columns:
    if col != "Player":
        player_metrics_with_avg[col] = player_metrics_with_avg[col].round(2).astype(str) + "%"


# 4b. Add IOC (country) column
# Extract IOC country code from Player
player_metrics_with_avg["ioc"] = player_metrics_with_avg["Player"].str.extract(r"\[(.*?)\]")

# Remove [XXX] from Player column and clean spaces
player_metrics_with_avg["Player"] = (
    player_metrics_with_avg["Player"]
    .str.replace(r"\[.*?\]", "", regex=True)
    .str.strip()
)


# Save
player_metrics_with_avg.to_csv("final_tables_for_tableau/scatterplot_metrics.csv", index=False)